## core

In [1]:
#| default_exp core

In [2]:
from __future__ import annotations
import duckdb
from duckdb import DuckDBPyConnection
from duckdb import DuckDBPyRelation
from typing import List, Dict, Optional, Union, Any, Tuple, Set
from fastcore.all import store_attr, patch, L
import numpy as np
import pandas as pd
from dataclasses import field, make_dataclass

from functools import wraps, partial

### Wrapping DuckDB Python API

When you `import duckdb`, there are two main concepts that you have to handle:


A *DuckDBPyConnection* represents a connection to a DuckDB database in a file or memory

In [3]:
db = duckdb.connect('../data/chinook.duckdb')
db

A DuckDBPyRelation represents the result of a query. It is a table-like object that can be queried further. 

In [4]:
db.sql(f"SELECT table_name FROM duckdb_tables()")

┌───────────────┐
│  table_name   │
│    varchar    │
├───────────────┤
│ Album         │
│ Artist        │
│ Customer      │
│ Employee      │
│ Genre         │
│ Invoice       │
│ InvoiceLine   │
│ MediaType     │
│ Playlist      │
│ PlaylistTrack │
│ Track         │
├───────────────┤
│    11 rows    │
└───────────────┘

In [5]:
type(db.sql(f"SELECT table_name FROM duckdb_tables()"))

duckdb.duckdb.DuckDBPyRelation

In [6]:
#| export
@patch(as_prop=True)
def table_names(self: DuckDBPyConnection): 
    ''' Tables in the current schema ''' 
    return [r[0] for r in self.sql(f"SELECT table_name FROM duckdb_tables() WHERE schema_name = current_schema()").fetchall()]
@patch(as_prop=True)
def view_names(self: DuckDBPyConnection): 
    ''' Views in the current schema ''' 
    return [r[0] for r in self.sql(f"SELECT view_name FROM duckdb_views() WHERE schema_name = current_schema() and internal = False").fetchall()]
@patch(as_prop=True)
def function_names(self: DuckDBPyConnection): 
    ''' Functions in the current schema ''' 
    return [r[0] for r in self.sql(f"SELECT function_name FROM duckdb_functions() WHERE schema_name = current_schema() and internal=False").fetchall()]

In [7]:
db.table_names, db.view_names, db.function_names

(['Album',
  'Artist',
  'Customer',
  'Employee',
  'Genre',
  'Invoice',
  'InvoiceLine',
  'MediaType',
  'Playlist',
  'PlaylistTrack',
  'Track'],
 ['a_view', 'temp_view'],
 [])

In [8]:
#| export

@patch(as_prop=True) # alias for alias
def name(self:DuckDBPyRelation): return self.alias

@patch
def __getitem__(self:DuckDBPyRelation, idxs) -> DuckDBPyRelation:
    return self.select(*idxs) if isinstance(idxs, Union[List, Set, Tuple]) else self.select(idxs)
@patch 
def to_recs(self:DuckDBPyRelation) -> List[Dict[str, Any]]:
    '''The relation as a list of records'''
    return self.df().to_dict(orient='records')

@patch 
def q(self:DuckDBPyConnection, *args, **kwargs) -> List[Dict[str, Any]]:
    '''Run a query and return the result as a list of records'''
    return self.sql(*args, **kwargs).to_recs()

In [9]:
db.table('Album')['AlbumId', 'Title'].limit(2).to_recs()

[{'AlbumId': 1, 'Title': 'For Those About To Rock We Salute You'},
 {'AlbumId': 2, 'Title': 'Balls to the Wall'}]

In [10]:
#| export

@patch
def datamodel(self: DuckDBPyConnection, table_name:str) ->List[Dict]:
    ''' Returns the data model of a table or view. 
    The datamodel contains the names, types, nullable status, default value and
    primary key status of the relation columns.'''
    
    return [{'name': r[1], 'type': r[2], 'nullable': not r[3], 'default': r[4], 'pk': r[5]} 
            for r in self.sql(f"PRAGMA table_info='{table_name}'").fetchall()]

In [11]:
db.datamodel('Artist')


[{'name': 'ArtistId',
  'type': 'INTEGER',
  'nullable': False,
  'default': None,
  'pk': True},
 {'name': 'Name',
  'type': 'VARCHAR',
  'nullable': True,
  'default': None,
  'pk': False}]

In [12]:
#| export

from dataclasses import field, make_dataclass
def convertTypes(s:str)->type:
    ''' Convert DuckDB types to Python and Numpy types'''
    d = {
        # Built-in types
        'BOOLEAN': bool,
        'BLOB': bytearray,  # For bytes, bytearray can be used in Python
        'DOUBLE': float,
        'BIGINT': int,
        'VARCHAR': str,
    
        # NumPy DTypes
        'FLOAT': float,
        'DOUBLE': float,
        'SMALLINT': np.int16,
        'INTEGER': np.int32,
        'TINYINT': np.int8,
        'USMALLINT': np.uint16,
        'UINTEGER': np.uint32,
        'UBIGINT': np.uint64,
        'UTINYINT': np.uint8
    }
    if s in d: return d[s]
    if s[:7]=='DECIMAL': return float
    return None

@patch
def dataclass(self: DuckDBPyConnection, 
              table_name:str, # table or view name
              pref='', # prefix to add to the field names
              suf='' # suffix to add to the field names
              ) -> type:
   '''Creates a `dataclass` type from a table or view in the database.'''
   fields = self.datamodel(table_name)
   fields = [(pref+f['name']+suf, convertTypes(f['type']) if not f['nullable'] else convertTypes(f['type'])|None , field(default=f['default'])) for f in fields]
   return make_dataclass(table_name, fields)

In [13]:
from fastcore.xtras import hl_md, dataclass_src

artist_dc = db.dataclass('Artist')
src = dataclass_src(artist_dc)
hl_md(src, 'python')

```python
@dataclass
class Artist:
    ArtistId: int32 = None
    Name: str | None = None

```

In [14]:
acdc = db.sql(f"select * from artist where artist.Name like 'AC/%'").df().to_dict(orient='records')
acdc

[{'ArtistId': 1, 'Name': 'AC/DC'}]

In [15]:
acdc_object = artist_dc(**acdc[0])
acdc_object

Artist(ArtistId=1, Name='AC/DC')

In [16]:
#| export

@patch
def _metadata(self: DuckDBPyConnection,name:str, type:str='table') -> Dict:
    table = (self.table(name) if type == 'table' else self.view(name))
    query = f"select comment from duckdb_{type}s() "
    where = f"where {type}_name = '{name}' and "
    where +=f"schema_name = current_schema() and internal = False"
    meta = self.sql(query+where).to_recs()[0]
    meta['row_count'] = table.shape[0]
    meta['col_count'] = table.shape[1]  
    return meta
@patch
def metadata(self: DuckDBPyConnection, name:str) -> Dict:
    '''Metadata of a table or view'''
    if name in self.table_names: return self._metadata(name, 'table')
    else: return self._metadata(name, 'view')

In [17]:
db.metadata('a_view')

{'comment': 'Just a view', 'row_count': 3503, 'col_count': 9}

In [18]:
album = db.sql("select * from Album")
album['AlbumId', 'Title']

┌─────────┬─────────────────────────────────────────────────────────────────────┐
│ AlbumId │                                Title                                │
│  int32  │                               varchar                               │
├─────────┼─────────────────────────────────────────────────────────────────────┤
│       1 │ For Those About To Rock We Salute You                               │
│       2 │ Balls to the Wall                                                   │
│       3 │ Restless and Wild                                                   │
│       4 │ Let There Be Rock                                                   │
│       5 │ Big Ones                                                            │
│       6 │ Jagged Little Pill                                                  │
│       7 │ Facelift                                                            │
│       8 │ Warner 25 Anos                                                      │
│       9 │ Play

In [19]:
album.columns, album.types, album.name

(['AlbumId', 'Title', 'ArtistId'],
 [INTEGER, VARCHAR, INTEGER],
 'unnamed_relation_f494bc5b3f3803fb')

In [20]:
#column_name, column_type, notnull, dflt_value, pk
db.sql(f"PRAGMA table_info=album").fetchall() 

[(0, 'AlbumId', 'INTEGER', True, None, True),
 (1, 'Title', 'VARCHAR', True, None, False),
 (2, 'ArtistId', 'INTEGER', True, None, False)]

In [21]:
#add comment to table information_schema in duckdb
db.sql("COMMENT ON TABLE Album IS 'Album table'")
db.metadata('Album')

{'comment': 'Album table', 'row_count': 347, 'col_count': 3}

In [22]:
db.sql("COMMENT ON VIEW a_view IS 'Just a view'")
db.metadata('a_view')

{'comment': 'Just a view', 'row_count': 3503, 'col_count': 9}

In [23]:
from fastcore.xtras import hl_md, dataclass_src

artist_dc = db.dataclass('Artist')
src = dataclass_src(artist_dc)
hl_md(src, 'python')
acdc = db.q(f"select * from artist where artist.Name like 'AC/%'")
acdc_object = artist_dc(**acdc[0])
acdc_object

Artist(ArtistId=1, Name='AC/DC')

#### New types: Database, Table and View

DuckDB API has no concept of a Database, only of a connection that may envolve different attached databases (or catalogs) and schemas.

Here we define a Database as a wrapper to a connection focused on a specific catalog and schema.

In [24]:
#| export

def noop(*args, **kwargs): return None
def identity(x): return x

In [25]:
#| export

class _Getter: 
    """ A Getter utility check https://github.com/AnswerDotAI/fastlite """
    def __init__(self, name:str='', type:str='', dir:List=[], get=noop): store_attr()    
    def __dir__(self): return self.dir
    def __str__(self): return ", ".join(dir(self))
    def __repr__(self): return f"{self.type}s: {str(self)}"
    def __contains__(self, s:str): return s in dir(self)
    def __getitem__(self, k): return self.get(k)
    def __getattr__(self, k):
        if k[0]!='_': return self.get(k)
        else: raise AttributeError 

In [26]:
#| export
def custom_dir(c, add): return dir(type(c)) + list(c.__dict__.keys()) + add

class Database:
    ''' A Database object that wraps a DuckDB connection and provides access to tables, views, functions and secrets.'''
    def _current(self: DuckDBPyConnection): return self.sql('select current_catalog, current_schema').fetchone()

    def __init__(self, *args, **kwargs):
        self.conn = duckdb.connect(*args, **kwargs) # original connection object is often called
        self.catalog, self.schema = self.conn.sql('select current_catalog, current_schema').fetchone()
        self.tables =  {}
        self.views = {}
        self.xtra = ['query', 'cursor', 'execute', 'fetchone', 'fetchall', 'sql', 'close', 'table_names', 'view_names', 'dataclass', 'metadata']

    def __dir__(self:Database): return custom_dir(self, self.xtra)  
    def __repr__(self): return f'{self.__class__.__name__} ({self.catalog}::{self.schema})'

    def __getattr__(self:Database, k):
        if k in self.xtra: return getattr(self.conn, k)
        if not isinstance(k, str) and k[0]=='_': raise AttributeError
        return self.table(k, include_views=True)        
          
    def __getitem__(self, k): return self.table(k)
    
    
    def table(self, k):
        if isinstance(k, Union[List, Set, Tuple]): return [self.table(ki) for ki in k]
        if not isinstance(k,str): raise AttributeError
        if self.conn.table(k): 
            tbl = Table(self.conn.table(k), self, k)
            self.tables= {**self.tables, **{k: tbl}}
            return tbl
    
    def view(self, k):
        if isinstance(k, Union[List, Set, Tuple]): return [self.view(ki) for ki in k]
        if not isinstance(k,str): raise AttributeError
        if self.conn.view(k): 
            vw = View(rel=self.conn.view(k), db=self, name=k)
            self.views= {**self.views, **{k: vw}}
            return vw
    
        
    @property # tables
    def t(self:Database): return _Getter('Tables', 'Table', self.conn.table_names, self.table)
    @property # views
    def v(self:Database): return _Getter('Views', 'View', self.conn.view_names, self.view)
    @property # functions
    def fns(self:Database): raise NotImplementedError
    # def fns(self:Database): return _Getter(self, f"SELECT function_name FROM duckdb_functions() WHERE schema_name = '{self.schema}' and internal = False")
    @property # secrets
    def shh(self:Database): raise NotImplementedError
    # def shh(self:Database): return _Getter(self, f"SELECT name FROM duckdb_secrets()")

    def q(self:Database, query:str): return self.conn.sql(query).to_recs()
    

In [27]:
#| export
class Table:
    ''' A Table object that wraps a DuckDBPyRelation and provides access to columns, metadata and dataclass.'''
    def __init__(self, rel:duckdb.DuckDBPyRelation, db:Database, name:str):
        store_attr()
        self.type='Table'
        self.cls = db.dataclass(name)
        self.meta = db.metadata(name)
        self.rel = self.rel.set_alias(name)    
        self.c = _Getter('Columns', 'Column', rel.columns, rel.select)
    def __dir__(self): return custom_dir(self, [s for s in dir(self.rel) if s[0]!= '_'])
    def __getattr__(self, k): 
        if k[0]=='_': raise AttributeError
        return getattr(self.rel, k)
    def __getitem__(self, idx): return self.rel[idx]
    def __str__(self): return f'"{self.db.schema}"."{self.name}"'
    def __repr__(self): return f'<{self.__class__.__name__}  {self.name} {dir(self.c)}>'
    def _repr_markdown_(self):
        markdown = f"**{self.__class__.__name__}: {self.name}** ({self.meta['row_count']} rows, {self.meta['col_count']} cols)\n"
        if self.meta['comment']: markdown += f"> {self.meta['comment']}"
        markdown = (self.rel.df()).head(10).to_markdown(index=False)
        return markdown
        
class View(Table):
    def __init__(self, rel:duckdb.DuckDBPyRelation, db:Database, name:str):
        super().__init__(rel, db, name) 
        self.type = 'View'
    def __getattr__(self, k): # must be overriden otherwise doesn't work
        if k[0]=='_': raise AttributeError
        return getattr(self.rel, k)

In [28]:
db.close() # duckdb connection

In [29]:
db = Database('../data/chinook.duckdb') # now as a Database
db

Database (chinook::main)

In [30]:
db.t # autocomplete of table names with tab inside Jupyter and IPython

Tables: Album, Artist, Customer, Employee, Genre, Invoice, InvoiceLine, MediaType, Playlist, PlaylistTrack, Track

::: 
 
![Autocomplete in Jupyter](images/autocomplete.png){.lightbox}

In [31]:
a = db.t.Album
a

|   AlbumId | Title                                 |   ArtistId |
|----------:|:--------------------------------------|-----------:|
|         1 | For Those About To Rock We Salute You |          1 |
|         2 | Balls to the Wall                     |          2 |
|         3 | Restless and Wild                     |          2 |
|         4 | Let There Be Rock                     |          1 |
|         5 | Big Ones                              |          3 |
|         6 | Jagged Little Pill                    |          4 |
|         7 | Facelift                              |          5 |
|         8 | Warner 25 Anos                        |          6 |
|         9 | Plays Metallica By Four Cellos        |          7 |
|        10 | Audioslave                            |          8 |

#### Export

In [32]:
#| hide
import nbdev; nbdev.nbdev_export()